## Setup

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import vgg19

from img_utils import (
    preprocess_image,
    deprocess_image
)
from losses import compute_loss_and_grads

# base_image_name = 'paris.jpg'
# base_image_url = 'https://i.imgur.com/F28w3Ac.jpg'
base_image_name = 'content.jpg'
# base_image_url = 'https://i.imgur.com/IGUM23Y.jpeg'
base_image_url = 'https://i.imgur.com/FPHjb7s.jpg'

# style_image_name = 'starry_night.jpg'
# style_image_url = "https://i.imgur.com/9ooB60I.jpg"
style_image_name = 'style.jpg'
# style_image_url = 'https://i.imgur.com/mtZi6rK.jpeg'
# style_image_url = 'https://i.imgur.com/tsxpI4N.jpeg'
style_image_url = 'https://i.imgur.com/y9dH2bJ.jpeg'

base_image_path = keras.utils.get_file(base_image_name, base_image_url)
style_reference_image_path = keras.utils.get_file(style_image_name, style_image_url)
# result_prefix = 'paris_generated'
result_prefix = 'generated'

# Weights of the different loss components
total_variation_weight = 1e-6
style_weight = 1e-6
content_weight = 2.5e-8

# Dimensions of the generated picture.
width, height = keras.preprocessing.image.load_img(base_image_path).size
img_nrows = 400
img_ncols = int(width * img_nrows / height)

2021-08-09 15:17:09.455814: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-09 15:17:09.455845: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Display initial images

In [ ]:
from IPython.display import Image, display

display(Image(base_image_path))
display(Image(style_reference_image_path))

## Feature Extraction Model

In [2]:
# Build a VGG19 model loaded with pre-trained ImageNet weights
model = vgg19.VGG19(weights="imagenet", include_top=False)

# Get the symbolic outputs of each "key" layer (we gave them unique names).
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

# Set up a model that returns the activation values for every layer in
# VGG19 (as a dict).
feature_extractor = keras.Model(inputs=model.inputs, outputs=outputs_dict)

2021-08-09 15:17:16.459426: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-08-09 15:17:16.459479: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-08-09 15:17:16.459509: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ivankras-ThinkPad-X1-Yoga-1st): /proc/driver/nvidia/version does not exist
2021-08-09 15:17:16.459859: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


80142336/80134624 [==============================] - 52s 1us/step


## Style Transfer Setup

In [ ]:
# List of layers to use for the style loss.
style_layer_names = [
    "block1_conv1",
    "block2_conv1",
    "block3_conv1",
    "block4_conv1",
    "block5_conv1",
]
# The layer to use for the content loss.
content_layer_name = "block5_conv2"

## Training

In [ ]:
optimizer = keras.optimizers.SGD(
    keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=100.0, decay_steps=100, decay_rate=0.96
    )
)

base_image = preprocess_image(base_image_path, img_nrows, img_ncols)
style_reference_image = preprocess_image(style_reference_image_path, img_nrows, img_ncols)
combination_image = tf.Variable(preprocess_image(base_image_path, img_nrows, img_ncols))

iterations = 4000
for i in range(1, iterations + 1):
    loss, grads = compute_loss_and_grads(
        combination_image, base_image, style_reference_image,
        feature_extractor, style_layer_names, content_layer_name,
        content_weight, style_weight, total_variation_weight,
        img_nrows, img_ncols
    )
    optimizer.apply_gradients([(grads, combination_image)])
    if i % 100 == 0:
        print("Iteration %d: loss=%.2f" % (i, loss))
        img = deprocess_image(combination_image.numpy(), img_nrows, img_ncols)
        fname = result_prefix + "_at_iteration_%d.png" % i
        keras.preprocessing.image.save_img(fname, img)

In [ ]:
# display(Image(result_prefix + "_at_iteration_1000.png"))
# display(Image(result_prefix + "_at_iteration_2000.png"))
# display(Image(result_prefix + "_at_iteration_3000.png"))
display(Image(result_prefix + "_at_iteration_4000.png"))